# Puzzle 1

--- Day 2: Gift Shop ---

You get inside and take the elevator to its only other stop: the gift shop. "Thank you for visiting the North Pole!" gleefully exclaims a nearby sign. You aren't sure who is even allowed to visit the North Pole, but you know you can access the lobby through here, and from there you can access the rest of the North Pole base.

As you make your way through the surprisingly extensive selection, one of the clerks recognizes you and asks for your help.

As it turns out, one of the younger Elves was playing on a gift shop computer and managed to add a whole bunch of invalid product IDs to their gift shop database! Surely, it would be no trouble for you to identify the invalid product IDs for them, right?

They've even checked most of the product ID ranges already; they only have a few product ID ranges (your puzzle input) that you'll need to check. For example:

11-22,95-115,998-1012,1188511880-1188511890,222220-222224,
1698522-1698528,446443-446449,38593856-38593862,565653-565659,
824824821-824824827,2121212118-2121212124

(The ID ranges are wrapped here for legibility; in your input, they appear on a single long line.)

The ranges are separated by commas (,); each range gives its first ID and last ID separated by a dash (-).

Since the young Elf was just doing silly patterns, you can find the invalid IDs by looking for any ID which is made only of some sequence of digits repeated twice. So, 55 (5 twice), 6464 (64 twice), and 123123 (123 twice) would all be invalid IDs.

None of the numbers have leading zeroes; 0101 isn't an ID at all. (101 is a valid ID that you would ignore.)

Your job is to find all of the invalid IDs that appear in the given ranges. In the above example:

    11-22 has two invalid IDs, 11 and 22.
    95-115 has one invalid ID, 99.
    998-1012 has one invalid ID, 1010.
    1188511880-1188511890 has one invalid ID, 1188511885.
    222220-222224 has one invalid ID, 222222.
    1698522-1698528 contains no invalid IDs.
    446443-446449 has one invalid ID, 446446.
    38593856-38593862 has one invalid ID, 38593859.
    The rest of the ranges contain no invalid IDs.

Adding up all the invalid IDs in this example produces 1227775554.

What do you get if you add up all of the invalid IDs?


In [19]:
from typing import Callable

In [15]:
def parse_ranges(input_line: str):
    input = [input_range.split('-') for input_range in input_line.split(',')]
    input_ranges = []
    for input_range in input:
        input_ranges.append([int(num) for num in input_range])
    return input_ranges

with open('../data/day2.txt', 'r') as input_fle:
    input = parse_ranges(input_fle.readline())
    

In [17]:
test_ranges = parse_ranges(
    '11-22,95-115,998-1012,1188511880-1188511890,222220-222224,1698522-1698528,446443-446449,38593856-38593862,565653-565659,824824821-824824827,2121212118-2121212124'
)

In [31]:
class IDValidator:
    ranges: str
    rule: Callable

    def __init__(self, ranges, rule = lambda x: len(x)%2 == 0 and x[:len(x)//2] == x[len(x)//2:]):
        self.ranges = ranges
        self.rule = rule

    def find_invalid_ids(self):
        invalid_ids = [
            id for interval in self.ranges for id in range(interval[0], interval[1]+1)
                if self.rule(str(id))
        ]
        return invalid_ids
    
    def find_invalid_sum(self):
        return sum(self.find_invalid_ids())
    

In [32]:
test = IDValidator(test_ranges)
test.find_invalid_sum()
# 1227775554
# success!

1227775554

In [33]:
validator = IDValidator(input)
validator.find_invalid_sum()
# 23701357374
# success!

23701357374

# Puzzle 2
--- Part Two ---

The clerk quickly discovers that there are still invalid IDs in the ranges in your list. Maybe the young Elf was doing other silly patterns as well?

Now, an ID is invalid if it is made only of some sequence of digits repeated at least twice. So, 12341234 (1234 two times), 123123123 (123 three times), 1212121212 (12 five times), and 1111111 (1 seven times) are all invalid IDs.

From the same example as before:

    11-22 still has two invalid IDs, 11 and 22.
    95-115 now has two invalid IDs, 99 and 111.
    998-1012 now has two invalid IDs, 999 and 1010.
    1188511880-1188511890 still has one invalid ID, 1188511885.
    222220-222224 still has one invalid ID, 222222.
    1698522-1698528 still contains no invalid IDs.
    446443-446449 still has one invalid ID, 446446.
    38593856-38593862 still has one invalid ID, 38593859.
    565653-565659 now has one invalid ID, 565656.
    824824821-824824827 now has one invalid ID, 824824824.
    2121212118-2121212124 now has one invalid ID, 2121212121.

Adding up all the invalid IDs in this example produces 4174379265.

What do you get if you add up all of the invalid IDs using these new rules?


In [63]:
#All we have to do is define a new callable to pass to IDValidator representing the new rule.
def new_rule(id: str):
    for ptr in range(1,len(id)//2+1):
        substr = id[:ptr]
        #If the string consists entirely of repetitions of this substring, splitting by it will yield a list of empty strings.
        #String splitting is efficient in Python, so while hacky I think this is fine.
        if len(''.join(id.split(substr))) == 0:
            return True
    return False

In [65]:
test_new_rule = IDValidator(test_ranges, rule = new_rule)
test_new_rule.find_invalid_sum()
# 4174379265
# success!

4174379265

In [66]:
validator_new_rule = IDValidator(input, rule = new_rule)
validator_new_rule.find_invalid_sum()
# 34284458938
# success!

34284458938

In [67]:
# More efficient
def new_rule_v2(id: str):
    if not id:
        return False
    return id in (id + id)[1:-1]

In [ ]:
validator_new_rule_v2 = IDValidator(input, rule = new_rule_v2)
validator_new_rule_v2.find_invalid_sum()
# 0.4s vs. 2.0s for v1

34284458938

# Reflection

Much easier than day 1, to me, or maybe I'm warmed up now. 

My secret challenge in doing these is to write a solution for Puzzle 1 that will also solve Puzzle 2 with minimal modifications. Today I was successful at predicting the flexibility required for part 2: changing the rule for flagging invalid IDs. 

Neither rule was too tricky to implement today, though it took me a bit of thought to come up with the more efficient v2 version of the substring detection for puzzle 2. 